# 🚀 **JLSatiro Clipper AI - V23.0 Titanium Edition**

Bem-vindo ao **JLSatiro Clipper AI**, a plataforma definitiva para criação de cortes virais automatizados.

### ⚡ **Recursos Titanium:**
- **Smart Crop V2.5**: Rastreamento facial ultra-preciso com MediaPipe (Fixed).
- **Hyper-Speed Transcription**: Transcrição GPU acelerada com Faster-Whisper.
- **Auto-Viral**: Geração automática de Thumbnails e Hooks.
- **100% Local**: Otimizado para rodar localmente no Colab com zero delay.
- **Titanium Vault**: Persistência total (Pasta Aberta) no Google Drive. Zero Instalação.

---

In [ ]:
#@title 1. 🛠️ Instalação do Sistema (Titanium Vault)
#@markdown O sistema usará o Drive como espelho. Se já instalado, copia para o SSD Local e inicia.

import os
import subprocess
import sys
import shutil
import time

print("💎 [JLSatiro] Iniciando Setup Titanium V23.0...")

# --- DRIVE MOUNT & VAULT CONFIG ---
print("☁️ [0/5] Conectando Drive...\n")
try:
    from google.colab import drive
    drive.mount('/content/drive')
    
    CONFIG_DIR = "/content/drive/MyDrive/JLSatiro_Config"
    DRIVE_LIB_PATH = os.path.join(CONFIG_DIR, "titanium_libs_raw")
    LOCAL_LIB_PATH = "/content/titanium_libs"
    
    os.makedirs(CONFIG_DIR, exist_ok=True)
    print(f"   ✅ Drive Conectado! Espelho em: {DRIVE_LIB_PATH}")
except Exception as e:
    print(f"   ⚠️ Aviso: Falha no Drive ({e}). Modo volátil (RAM) ativado.")
    CONFIG_DIR = None
    DRIVE_LIB_PATH = None
    LOCAL_LIB_PATH = "/content/titanium_libs"

# Configurar Path Python
if LOCAL_LIB_PATH not in sys.path:
    sys.path.insert(0, LOCAL_LIB_PATH)
os.environ["PYTHONPATH"] = f"{LOCAL_LIB_PATH}:{os.environ.get('PYTHONPATH', '')}"

# --- VAULT CHECK ---
vault_restored = False

# Verificando se o espelho existe no Drive
if DRIVE_LIB_PATH and os.path.exists(DRIVE_LIB_PATH) and len(os.listdir(DRIVE_LIB_PATH)) > 10 and not os.path.exists(".setup_done"):
    print(f"\n⚡ [VAULT] Bibliotecas encontradas no Drive! Sincronizando para SSD Local... (Isso pode custar uns segundos para garantir performance)")
    try:
        # Optimization: Use 'cp -a' for fast local sync
        if os.path.exists(LOCAL_LIB_PATH): shutil.rmtree(LOCAL_LIB_PATH)
        
        # Using rsync or cp for folder sync
        print("   📂 Copiando para SSD (/content)... Aguarde...")
        subprocess.run(f"cp -a '{DRIVE_LIB_PATH}/.' '{LOCAL_LIB_PATH}/'", shell=True)
        
        # System Deps Restore (Always fast)
        print("   📦 Restaurando Drivers de Vídeo...")
        subprocess.run("apt-get update -qq && apt-get install -y ffmpeg imagemagick libsndfile1 sox > /dev/null", shell=True)
        
        # Fix Magick
        if os.path.exists("/etc/ImageMagick-6/policy.xml"):
             subprocess.run("sed -i 's/none/read,write/g' /etc/ImageMagick-6/policy.xml", shell=True)

        vault_restored = True
        with open(".setup_done", "w") as f: f.write("restored")
        print("   ✅ SISTEMA RESTAURADO E PRONTO!")
    except Exception as e:
        print(f"   ❌ Falha na Sincronização: {e}. Reinstalando...")

# --- FRESH INSTALL (If Drive is Empty) ---
if not vault_restored and not os.path.exists(".setup_done"):
    # 1. Verificar GPU
    print("\n🖥️ [1/4] Verificando Hardware...")
    try:
        subprocess.run(["nvidia-smi"], check=True)
        print("   ✅ GPU NVIDIA Detectada! Modo Turbo Ativado.")
    except:
        print("   ⚠️ AVISO: GPU não detectada.")

    # 2. Instalar Dependências do Sistema (APT)
    print("📦 [2/4] Instalando Motores de Sistema (FFmpeg, ImageMagick)...")
    subprocess.run("apt-get update -qq", shell=True)
    subprocess.run("apt-get install -y ffmpeg imagemagick libsndfile1 sox -qq", shell=True)

    # CORREÇÃO: ImageMagick Policy p/ MoviePy
    if os.path.exists("/etc/ImageMagick-6/policy.xml"):
        print("   🔧 Aplicando Patch no ImageMagick...")
        subprocess.run("sed -i 's/none/read,write/g' /etc/ImageMagick-6/policy.xml", shell=True)

    # 3. Instalar Bibliotecas Python (PIP) -> TARGET LOCAL_LIB_PATH
    print(f"🐍 [3/4] Instalando Bibliotecas Python em {LOCAL_LIB_PATH}...")
    os.makedirs(LOCAL_LIB_PATH, exist_ok=True)

    # GPU LLM
    print("   🧠 Instalando Llama.cpp Otimizado (GPU/CUDA)...")
    subprocess.run(f'pip install llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu121 --no-cache-dir --target {LOCAL_LIB_PATH}', shell=True)

    libraries = [
        "torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121",
        "faster-whisper>=1.0.0",
        "openai-whisper",
        "mediapipe>=0.10.14",
        "transformers",
        "instructor>=0.5.0,<1.0.0",
        "pydantic",
        "moviepy",
        "opencv-python-headless",
        "librosa",
        "soundfile",
        "imageio",
        "imageio-ffmpeg",
        "pillow",
        "pydub",
        "gradio>=4.0.0,<5.0.0",
        "flask",
        "flask-cors",
        "pytubefix>=6.0.0",
        "yt-dlp",
        "python-dotenv",
        "numpy<2.0.0",
        "pandas",
        "edge-tts"
    ]

    pip_cmd = f"{sys.executable} -m pip install --upgrade --no-cache-dir --target {LOCAL_LIB_PATH}"
    for lib in libraries:
        subprocess.run(f"{pip_cmd} \"{lib}\"", shell=True, stdout=subprocess.DEVNULL)

    print("   ✅ Dependências Python instaladas.")
    
    # 4. SYNC TO DRIVE (Espelhamento Raw)
    if DRIVE_LIB_PATH:
        print("💾 [VAULT] Espelhando Bibliotecas para o Drive (Pasta Aberta)...")
        print("   ⏳ Isso pode demorar alguns minutos na primeira vez (muitos arquivos)...")
        try:
             if not os.path.exists(DRIVE_LIB_PATH):
                 subprocess.run(f"cp -a '{LOCAL_LIB_PATH}/.' '{DRIVE_LIB_PATH}/'", shell=True)
                 print("   ✅ ESPELHAMENTO CONCLUÍDO!")
        except Exception as e:
             print(f"   ⚠️ Erro ao espelhar: {e}")

    with open(".setup_done", "w") as f: f.write("ok")

elif os.path.exists(".setup_done"):
    print("\n⚡ [CACHE] Sistema já ativo e configurado!")


# 5. CONFIG WORKSPACE (Always Sync Code)
print("📂 [5/5] Sincronizando Código & Credenciais...")
repo_url = "https://github.com/noviso123/JLSatiroClipperAI.git"
repo_dir = "/content/JLSatiroClipperAI"

if not os.path.exists(repo_dir):
    subprocess.run(f"git clone {repo_url} {repo_dir}", shell=True)
else:
    subprocess.run(f"cd {repo_dir} && git pull", shell=True)

# RESTORE CREDENTIALS
if CONFIG_DIR:
    files_to_sync = ["client_secret.json", "token.pickle", "cookies.txt"]
    for f_name in files_to_sync:
        src = os.path.join(CONFIG_DIR, f_name)
        dst = os.path.join(repo_dir, f_name)
        if os.path.exists(src):
             # Copy if newer
             if not os.path.exists(dst) or (os.path.getmtime(src) > os.path.getmtime(dst)):
                 shutil.copy(src, dst)
                 print(f"   ♻️ Credencial Restaurada: {f_name}")

# Criar pastas necessárias
os.makedirs(f"{repo_dir}/downloads", exist_ok=True)
os.makedirs(f"{repo_dir}/temp_work", exist_ok=True)
os.makedirs(f"{repo_dir}/backend/models", exist_ok=True)

print("\n✨ SETUP COMPLETO! Execute o próximo bloco.")

In [ ]:
#@title 2. 🚀 Iniciar JLSatiro Clipper AI
#@markdown Clique no play e aguarde. O app irá usar as bibliotecas carregadas do Vault.

import os
import sys
import shutil

# Configurar paths novamente para garantir
LOCAL_LIB_PATH = "/content/titanium_libs"
if os.path.exists(LOCAL_LIB_PATH) and LOCAL_LIB_PATH not in sys.path:
    sys.path.insert(0, LOCAL_LIB_PATH)

# Garantir que estamos no diretório correto
os.chdir("/content/JLSatiroClipperAI")

# Forçar patch do MediaPipe caso o repo não tenha atualizado ainda (Backup Safety)
video_engine_path = "backend/video_engine.py"
if os.path.exists(video_engine_path):
    with open(video_engine_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    if "from mediapipe import solutions" in content:
        print("🔧 Aplicando correção hotfix no MediaPipe (Backup)...")
        content = content.replace(
            "from mediapipe import solutions\n\n        mp_face_detection = solutions.face_detection",
            "import mediapipe as mp\n\n        mp_face_detection = mp.solutions.face_detection"
        )
        content = content.replace(
            "from mediapipe import solutions\n\n        mp_face = solutions.face_detection",
            "import mediapipe as mp\n\n        mp_face = mp.solutions.face_detection"
        )
        with open(video_engine_path, 'w', encoding='utf-8') as f:
            f.write(content)

try:
    # Executar App (Isso bloqueia até o user parar)
    !python app.py
except KeyboardInterrupt:
    pass
finally:
    # BACKUP PARA O DRIVE AO FECHAR
    CONFIG_DIR = "/content/drive/MyDrive/JLSatiro_Config"
    if os.path.exists(CONFIG_DIR):
        print("\n💾 [AUTO-BACKUP] Sincronizando estado final...")
        files_to_sync = ["client_secret.json", "token.pickle", "cookies.txt"]
        for f_name in files_to_sync:
            if os.path.exists(f_name):
                 dst = os.path.join(CONFIG_DIR, f_name)
                 if not os.path.exists(dst) or (os.path.getmtime(f_name) > os.path.getmtime(dst)):
                    shutil.copy(f_name, dst)
                    print(f"   ✅ Salvo: {f_name}")
        print("🏁 Finalizado!")